In [19]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
import string
import numpy as np
from sklearn.utils import shuffle
import collections
import re
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\damio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
def tokenize(str_data):
    data = str_data.lower()
    data = re.sub(r'\n', '', data)
    data = re.sub(r'[^A-Za-z\s]', '', data)
    data = data.split(' ')
    data = list(filter(lambda item: item != '', data))
    return data

In [21]:
books = []
with open (f"HP1.txt", 'r') as f:
    pages = []
    data = f.read()
    data = data.split('\n')
    for page in data:
        pages.append(tokenize(page))
    books.append(pages)
    f.close()

with open (f"HP2.txt", 'r') as f:
    pages = []
    data = f.read()
    data = data.split('\n')
    for page in data:
        pages.append(tokenize(page))
    books.append(pages)
    f.close()

In [34]:
#[book][page]

all_data = []
labels = []

for i, book in enumerate(books):
    for page in book:
        all_data.append(page)
        labels.append(i + 1)